In [6]:
from statsforecast.models import AutoARIMA
from statsmodels.tsa.stattools import acf 
import pandas as pd 
import lineapy
import requests 
import re 
import numpy as np
from numpy.linalg import svd
import altair as alt

In [1]:
#NBVAL_SKIP
!pip -q install lineapy~=0.2 scikit-learn pandas matplotlib

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 6.1.3 requires colorama>=0.4.5; sys_platform == "win32", but you have colorama 0.4.4 which is incompatible.
spacy 3.3.1 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.10.7 which is incompatible.
pycaret 3.0.0 requires sktime>=0.16.1, but you have sktime 0.15.1 which is incompatible.
openplayground 0.1.3 requires torch<3.0.0,>=2.0.0, but you have torch 1.13.1 which is incompatible.
openbb 2.4.0 requires charset-normalizer==2.1.1, but you have charset-normalizer 2.0.12 which is incompatible.
openbb 2.4.0 requires grpcio<2.0.0,>=1.51.1, but you have grpcio 1.48.1 which is incompatible.
openbb 2.4.0 requires ipython==8.5.0, but you have ipython 7.34.0 which is incompatible.
openbb 2.4.0 requires ipywidgets<9.0.0,>=8.0.2, but you have ipywidgets 7.7.4 which is incompatible.
openbb 2.4.0 r

In [7]:
%load_ext lineapy 
#%load_ext nb_black

The lineapy extension is already loaded. To reload it, use:
  %reload_ext lineapy


In [8]:
response = requests.get("https://www.eia.gov/petroleum/gasdiesel/xls/pswrgvwall.xls")
df = pd.read_excel(
    response.content,
    sheet_name="Data 12",
    index_col=0,
    skiprows=2,
    parse_dates=["Date"],
).rename(
    columns=lambda c: re.sub(
        "\(PADD 1[A-C]\)",
        "",
        c.replace("Weekly ", "").replace(
            " All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)",
            "",
        ),
    ).strip()
)
lineapy.save(df, "weekly_gas_price_data")

LineaArtifact(name='weekly_gas_price_data', _version=0)

In [9]:
df_long = (
    df.reset_index()
    .melt(id_vars=["Date"], var_name="region", value_name="price")
    .rename(columns={"Date": "week"})
    .sort_values(["region", "week"])
    .assign(
        # if we're missing one value, just use the last value
        # (happens twice)
        price=lambda x: x["price"].combine_first(x.groupby("region")["price"].shift(1)),
        # we'll forecast log(price) and then transform
        log_price=lambda x: np.log(x["price"]),
        # percentage price changes are approximately the difference in log(price)
        price_change=lambda x: (
            x["log_price"] - x.groupby("region")["log_price"].shift(1)
        ),
    )
    .query("price == price")  # filter out NAs
)

lineapy.save(df_long, "weekly_gas_price_data_long")
df_long.head()

,week,region,price,log_price,price_change
28735,2003-05-26,"Boston, MA",1.555,0.441476,NaN
28736,2003-06-02,"Boston, MA",1.547,0.436318,-0.005158
28737,2003-06-09,"Boston, MA",1.534,0.427879,-0.008439
28738,2003-06-16,"Boston, MA",1.549,0.437610,0.009731
28739,2003-06-23,"Boston, MA",1.544,0.434376,-0.003233


In [10]:
df_long.groupby("region")["price"].count().reset_index().pipe(alt.Chart).encode(
    x=alt.X("price", title="Cases"), y=alt.Y("region", sort=alt.SortField("price"))
).mark_bar()

alt.Chart(...)

In [11]:
df_long.groupby("week")["price"].count().reset_index().pipe(alt.Chart).encode(
    x="week", y=alt.Y("price", title="Count")
).mark_line()

alt.Chart(...)

In [12]:
df_long["price_change"].mean() * 52

0.04056149607595902

In [13]:
(
    df_long.query("price_change == price_change")
    .sample(5000)
    .pipe(alt.Chart)
    .transform_density("price_change")
    .encode(x="value:Q", y="density:Q")
    .mark_area()
)

alt.Chart(...)

In [14]:
all_regions = df_long["region"].unique().tolist()
lineapy.save(all_regions, "all_regions")
num_regions = len(all_regions)
num_regions

28

In [15]:
complete_case_date = (
    df_long.groupby("week")["price"]
    .count()
    .reset_index()
    .query(f"price == {num_regions}")["week"]
    .min()
).strftime("%Y-%m-%d")
complete_case_date

'2003-05-26'

In [16]:
(
    df_long.groupby("region")["price_change"]
    .mean()
    .reset_index()
    .assign(annual_price_change=lambda x: x["price_change"] * 52)
    .pipe(alt.Chart)
    .encode(
        x=alt.X("region", sort=alt.SortField("annual_price_change")),
        y=alt.Y("annual_price_change", title="Annual Price Growth"),
    )
    .mark_bar()
)

alt.Chart(...)

In [17]:
wide = (
    df_long.query(f"week > '{complete_case_date}'")[["week", "region", "price_change"]]
    .set_index("week")
    .pivot(columns="region", values="price_change")
)
matrix = wide.values
print(matrix.shape)
u, d, v = svd(matrix)

(1037, 28)


In [18]:
scree_plot = (
    pd.DataFrame({"eigenvalue": d, "index": np.arange(d.shape[0])})
    .pipe(alt.Chart)
    .encode(x="index", y="eigenvalue")
    .mark_point()
)

lineapy.save(scree_plot, "scree_plot")
scree_plot

alt.Chart(...)

In [19]:
components = pd.DataFrame(
    v, columns=[f"component_{i}" for i in range(v.shape[0])], index=wide.columns
).reset_index()

components_plot = (
    components.pipe(alt.Chart)
    .encode(x="component_0", y="component_1", text="region")
    .mark_text()
    .interactive()
)

lineapy.save(components_plot, "components_plot")
components_plot

alt.Chart(...)

In [20]:
region = "U.S."
auto_correlation = (
    df_long.query(f"region == '{region}'")
    .query("price_change == price_change")["price_change"]
    .pipe(acf)
)
acf_plot = (
    pd.DataFrame({"rho": auto_correlation, "lag": np.arange(auto_correlation.shape[0])})
    .pipe(alt.Chart, title=region)
    .encode(x="lag", y="rho")
    .mark_bar()
)
lineapy.save(acf_plot, "acf_plot2")
acf_plot

alt.Chart(...)

In [22]:
H = 13
CI = 80
width = 300
height = 250
region = "U.S."
cutoff_date = "2023-04-16"
plot_start_date = "2022-01-01"
plot_title = f"{region} (as of {cutoff_date})"

In [23]:
region_df = df_long.query(f"region == '{region}'")
train = region_df.query(f"week < '{cutoff_date}'")
m_aa = AutoARIMA()
m_aa.fit(train["log_price"].values)

AutoARIMA

In [24]:
raw_forecast = m_aa.predict(h=H, level=(CI,))
raw_forecast_exp = {key: np.exp(value) for key, value in raw_forecast.items()}
forecast = pd.DataFrame(raw_forecast_exp).assign(
    week=pd.date_range(train["week"].max(), periods=H, freq="W")
    + pd.Timedelta("7 days")
)
forecast = pd.concat(
    [
        forecast,
        train.tail(1)
        .rename(columns={"price": "mean"})
        .assign(**{f"lo-{CI}": lambda x: x["mean"], f"hi-{CI}": lambda x: x["mean"]}),
    ]
)
forecast.head()

,mean,lo-80,hi-80,week,region,log_price,price_change
0,3.755395,3.681131,3.831157,2023-04-23,NaN,NaN,NaN
1,3.783427,3.647836,3.924058,2023-04-30,NaN,NaN,NaN
2,3.798349,3.608179,3.998542,2023-05-07,NaN,NaN,NaN
3,3.806272,3.568323,4.060088,2023-05-14,NaN,NaN,NaN
4,3.810472,3.530554,4.112583,2023-05-21,NaN,NaN,NaN


In [25]:
uncertainty_plot = (
    forecast.pipe(alt.Chart, height=height, width=width)
    .encode(
        x="week",
        y=alt.Y(f"lo-{CI}", title="Price"),
        y2=alt.Y2(f"hi-{CI}", title="Price"),
    )
    .mark_area(opacity=0.2)
)

history_plot = (
    region_df.query(f"week >= '{plot_start_date}'")
    .pipe(alt.Chart, title=plot_title)
    .encode(x=alt.X("week", title="Week"), y=alt.Y("price", title="Price"))
    .mark_line()
)

forecast_plot = forecast.pipe(alt.Chart).encode(x="week", y="mean").mark_line()

cutoff_plot = (
    train.tail(1).pipe(alt.Chart).encode(x="week").mark_rule(strokeDash=[10, 2])
)

full_plot = uncertainty_plot + history_plot + forecast_plot + cutoff_plot
lineapy.save(full_plot, "gas_price_forecast")

LineaArtifact(name='gas_price_forecast', _version=0)

In [38]:
full_plot

alt.LayerChart(...)

<IPython.core.display.Javascript object>

In [26]:
forecast_region = lineapy.get_function(
    ["gas_price_forecast"],
    input_parameters=[
        "region",
        "cutoff_date",
        "H",
        "width",
        "height",
        "plot_start_date",
    ],
    reuse_pre_computed_artifacts=["weekly_gas_price_data_long"],
)

In [29]:
result = forecast_region(
    region="California", cutoff_date="2023-04-15", H=15, width=300, height=250
)
result["gas_price_forecast"]

alt.LayerChart(...)

In [30]:
plots = []
for region in all_regions:
    result = forecast_region(
        region=region, cutoff_date=cutoff_date, height=200, width=200
    )
    plots.append(result["gas_price_forecast"])

In [31]:
chart = alt.vconcat()
for i, plot in enumerate(plots):
    if i % 4 == 0:
        row = alt.hconcat()
        chart &= row
    row |= plot
chart

alt.VConcatChart(...)

In [32]:
lineapy.save(chart, "all_forecasts_plot")

LineaArtifact(name='all_forecasts_plot', _version=0)

In [33]:
lineapy.to_pipeline(
    ["gas_price_forecast", "weekly_gas_price_data", "weekly_gas_price_data_long"],
    dependencies={
        "gas_price_forecast": {"weekly_gas_price_data_long": {"weekly_gas_price_data"}}
    },
    pipeline_name="gas_price_forecast",
    output_dir="pipeline",
    framework="AIRFLOW",
    input_parameters=["region", "cutoff_date"],
)

Generated module file: pipeline\gas_price_forecast_module.py                   
Generated requirements file: pipeline\gas_price_forecast_requirements.txt      
Generated DAG file: pipeline\gas_price_forecast_dag.py                         
Generated Docker file: pipeline\gas_price_forecast_Dockerfile                  


WindowsPath('pipeline')

In [34]:
%pycat pipeline/gas_price_forecast_dag.py

import pathlib
import pickle

import gas_price_forecast_module
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago


def task_weekly_gas_price_data():
    df = gas_price_forecast_module.get_weekly_gas_price_data()

    if not pathlib.Path("/tmp").joinpath("gas_price_forecast").exists():
        pathlib.Path("/tmp").joinpath("gas_price_forecast").mkdir()
    pickle.dump(df, open("/tmp/gas_price_forecast/variable_df.pickle", "wb"))


def task_weekly_gas_price_data_long():
    df = pickle.load(open("/tmp/gas_price_forecast/variable_df.pickle", "rb"))

    df_long = gas_price_forecast_module.get_weekly_gas_price_data_long(df)

    if not pathlib.Path("/tmp").joinpath("gas_price_forecast").exists():
        pathlib.Path("/tmp").joinpath("gas_price_forecast").mkdir()
    pickle.dump(df_long, open("/tmp/gas_price_forecast/variable_df_long.pickle", "wb"))


def task_gas_price_forecast(cutoff_date, region):
    cutoff_dat

In [35]:
%pycat pipeline/gas_price_forecast_module.py

import argparse
import re

import altair as alt
import numpy as np
import pandas as pd
import requests
from numpy.linalg import svd
from statsforecast.models import AutoARIMA


def get_weekly_gas_price_data():
    response = requests.get(
        "https://www.eia.gov/petroleum/gasdiesel/xls/pswrgvwall.xls"
    )
    df = pd.read_excel(
        response.content,
        sheet_name="Data 12",
        index_col=0,
        skiprows=2,
        parse_dates=["Date"],
    ).rename(
        columns=lambda c: re.sub(
            "\(PADD 1[A-C]\)",
            "",
            c.replace("Weekly ", "").replace(
                " All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)",
                "",
            ),
        ).strip()
    )
    return df


def get_weekly_gas_price_data_long(df):
    df_long = (
        df.reset_index()
        .melt(id_vars=["Date"], var_name="region", value_name="price")
        .rename(columns={"Date": "week"})
        .sort_values(["region", 

In [36]:
%pycat pipeline/gas_price_forecast_requirements.txt

statsforecast==1.4.0
pandas==1.5.3
requests==2.28.2
re==2.2.1
numpy==1.22.0
altair==4.2.2
